In [1]:
import spacy
import pandas as pd
import numpy as np
import re
import string
import multiprocessing as mp
import spacy

# Load the English model
nlp = spacy.load("en_core_web_sm")

from functools import reduce
from operator import add
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\admin\anaconda3\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Set All Recommendation Model Parameters
N_topics = 50             # Number of Topics to Extract from corpora
N_top_docs = 200          # Number of top documents within each topic to extract keywords
N_top_words = 25          # Number of keywords to extract from each topic
N_docs_categorized = 2000 # Number of top documents within each topic to tag 
N_neighbor_window = 4     # Length of word-radius that defines the neighborhood for
                          # each word in the TextRank adjacency table

# Query Similarity Weights
w_title = 0.2
w_text = 0.3
w_categories = 0.5
w_array = np.array([w_title, w_text, w_categories])

# Recipe Stopwords: for any high volume food recipe terminology that doesn't contribute
# to the searchability of a recipe. This list must be manually created.
recipe_stopwords = ['cup','cups','ingredient','ingredients','teaspoon','teaspoons','tablespoon',
                   'tablespoons','C','F']

In [10]:
# Load the CSV file into a Pandas DataFrame
tokenized_text = pd.read_csv('tokenized_text.csv')

# Print the first row of the tokenized text
print(tokenized_text.iloc[0, 1])

Slow Cooker Chicken Dumplings skinless boneless chicken breast half tablespoon butter ounce can condense cream chicken soup onion finely dice ounce package refrigerate biscuit dough tear piece place chicken butter soup onion slow cooker fill water cover Cover cook hour High minute serve place tear biscuit dough slow cooker Cook dough long raw center


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming tokenized_text is your DataFrame with tokenized text
tokenized_text = pd.read_csv('tokenized_text.csv')

# Extracting the tokenized text from the DataFrame
tokenized_text = tokenized_text['0']

# Create the TfidfVectorizer
vectorizer = TfidfVectorizer(lowercase=True, ngram_range=(1, 1))

# Fit and transform your data
text_tfidf = vectorizer.fit_transform(tokenized_text)

# Get feature names
tfidf_words = vectorizer.get_feature_names_out()

print(text_tfidf.shape)
print(len(tfidf_words))


(122911, 49792)
49792


In [25]:
# Loading Data
allrecipes_raw = pd.read_json('recipes_raw_nosource_ar.json')
epicurious_raw = pd.read_json('recipes_raw_nosource_epi.json')
foodnetwork_raw = pd.read_json('recipes_raw_nosource_fn.json')

allrecipes = allrecipes_raw.copy().T.reset_index().drop(columns = ['index'])
epicurious = epicurious_raw.copy().T.reset_index().drop(columns = ['index'])
foodnetwork = foodnetwork_raw.copy().T.reset_index().drop(columns = ['index'])
recipes = pd.concat([allrecipes, epicurious, foodnetwork]).reset_index(drop=True) # Concat does not reset indices

# Cleaning
null_recs = recipes.copy().drop(columns = 'picture_link').T.isna().any()
rows_to_drop = recipes[null_recs].index
recipes = recipes.drop(index = rows_to_drop).reset_index(drop = True)

nc_ingred_index = [index for i, index in zip(recipes['ingredients'], recipes.index) if all(j.isdigit() or j in string.punctuation for j in i)]
nc_title_index = [index for i, index in zip(recipes['title'], recipes.index) if all(j.isdigit() or j in string.punctuation for j in i)]
nc_instr_index = [index for i, index in zip(recipes['instructions'], recipes.index) if all(j.isdigit() or j in string.punctuation for j in i)]

index_list = [nc_ingred_index, nc_title_index, nc_instr_index]

inds_to_drop = set(reduce(add, index_list))
print(len(inds_to_drop))
recipes = recipes.drop(index=inds_to_drop).reset_index(drop=True)
recipes.shape

empty_instr_ind = [index for i, index in zip(recipes['instructions'], recipes.index) if len(i) < 20]
recipes = recipes.drop(index = empty_instr_ind).reset_index(drop=True)

ingredients = []
for ing_list in recipes['ingredients']:
    clean_ings = [ing.replace('ADVERTISEMENT','').strip() for ing in ing_list]
    if '' in clean_ings:
        clean_ings.remove('')
    ingredients.append(clean_ings)
recipes['ingredients'] = ingredients

recipes['ingredient_text'] = ['; '.join(ingredients) for ingredients in recipes['ingredients']]
recipes['ingredient_text'].head()

recipes['ingredient_count'] = [len(ingredients) for ingredients in recipes['ingredients']]

all_text = recipes['title'] + ' ' + recipes['ingredient_text'] + ' ' + recipes['instructions']

def clean_text(documents):
    cleaned_text = []
    for doc in documents:
        doc = doc.translate(str.maketrans('', '', string.punctuation)) # Remove Punctuation
        doc = re.sub(r'\d+', '', doc) # Remove Digits
        doc = doc.replace('\n',' ') # Remove New Lines
        doc = doc.strip() # Remove Leading White Space
        doc = re.sub(' +', ' ', doc) # Remove multiple white spaces
        cleaned_text.append(doc)
    return cleaned_text

cleaned_text = clean_text(all_text)

# Testing Strategies and Code
nlp = spacy.load('en_core_web_sm')
' '.join([token.lemma_ for token in nlp(cleaned_text[2]) if not token.is_stop])

def text_tokenizer_mp(doc):
    tok_doc = ' '.join([token.lemma_ for token in nlp(doc) if not token.is_stop])
    return tok_doc

1551


'vectorizer = TfidfVectorizer(lowercase = True,\n                     ngram_range = (1,1))\n\ntext_tfidf = vectorizer.fit_transform(tokenized_text)'

In [26]:
# Renaming Data Dependencies
topic_transformed_matrix = text_nmf
root_text_data = cleaned_text

In [27]:
from itertools import repeat

#recipes['tag_list'] = [[] for i in repeat(None, recipes.shape[0])]

def topic_docs_4kwsummary(topic_document_scores, root_text_data):
    '''Gathers and formats the top recipes in each topic'''
    text_index = pd.Series(topic_document_scores).sort_values(ascending = False)[:N_top_docs].index
    text_4kwsummary = pd.Series(root_text_data)[text_index]
    return text_4kwsummary

def generate_filter_kws(text_list):
    '''Filters out specific parts of speech and stop words from the list of potential keywords'''
    parsed_texts = nlp(' '.join(text_list)) 
    kw_filts = set([str(word) for word in parsed_texts 
                if (word.pos_== ('NOUN' or 'ADJ' or 'VERB'))
                and word.lemma_ not in recipe_stopwords])
    return list(kw_filts), parsed_texts

def generate_adjacency(kw_filts, parsed_texts):
    '''Tabulates counts of neighbors in the neighborhood window for each unique word'''
    adjacency = pd.DataFrame(columns=kw_filts, index=kw_filts, data = 0)
    for i, word in enumerate(parsed_texts):
        if any ([str(word) == item for item in kw_filts]):
            end = min(len(parsed_texts), i+N_neighbor_window+1) # Neighborhood Window Utilized Here
            nextwords = parsed_texts[i+1:end]
            inset = [str(x) in kw_filts for x in nextwords]
            neighbors = [str(nextwords[i]) for i in range(len(nextwords)) if inset[i]]
            if neighbors:
                adjacency.loc[str(word), neighbors] += 1
    return adjacency
                
def generate_wordranks(adjacency):
    '''Runs TextRank on adjacency table'''
    nx_words = nx.from_numpy_matrix(adjacency.values)
    ranks=nx.pagerank(nx_words, alpha=.85, tol=.00000001)
    return ranks

def generate_tag_list(ranks):
    '''Uses TextRank ranks to return actual key words for each topic in rank order'''
    rank_values = [i for i in ranks.values()]
    ranked = pd.DataFrame(zip(rank_values, list(kw_filts))).sort_values(by=0,axis=0,ascending=False)
    kw_list = ranked.iloc[:N_top_words,1].to_list()
    return kw_list

# Master Function utilizing all above functions
def generate_tags(topic_document_scores, root_text_data):
    text_4kwsummary = topic_docs_4kwsummary(topic_document_scores, root_text_data)
    kw_filts, parsed_texts = generate_filter_kws(text_4kwsummary)
    adjacency = generate_adjacency(kw_filts, parsed_texts)
    ranks = generate_wordranks(adjacency)
    kw_list = generate_tag_list(ranks)
    return kw_list

def generate_kw_index(topic_document_scores):
    kw_index = pd.Series(topic_document_scores).sort_values(ascending = False)[:N_docs_categorized].index
    return kw_index


In [28]:
recipes = pd.read_csv('tagged_recipes_df.csv')

In [30]:
recipes.shape

(122911, 8)

In [31]:
recipes

,Unnamed: 0,title,ingredients,instructions,picture_link,ingredient_text,ingredient_count,tag_list
0,0,Slow Cooker Chicken and Dumplings,"['4 skinless, boneless chicken breast halves',...","Place the chicken, butter, soup, and onion in ...",55lznCYBbs2mT8BTx6BTkLhynGHzM.S,"4 skinless, boneless chicken breast halves; 2 ...",5,NaN
1,1,Awesome Slow Cooker Pot Roast,['2 (10.75 ounce) cans condensed cream of mush...,"In a slow cooker, mix cream of mushroom soup, ...",QyrvGdGNMBA2lDdciY0FjKu.77MM0Oe,2 (10.75 ounce) cans condensed cream of mushro...,4,"grease, paperline, hours, glaze, cranberries, ..."
2,2,Brown Sugar Meatloaf,"['1/2 cup packed brown sugar', '1/2 cup ketchu...",Preheat oven to 350 degrees F (175 degrees C)....,LVW1DI0vtlCrpAhNSEQysE9i/7rJG56,1/2 cup packed brown sugar; 1/2 cup ketchup; 1...,10,NaN
3,3,Best Chocolate Chip Cookies,"['1 cup butter, softened', '1 cup white sugar'...",Preheat oven to 350 degrees F (175 degrees C)....,0SO5kdWOV94j6EfAVwMMYRM3yNN8eRi,"1 cup butter, softened; 1 cup white sugar; 1 c...",11,"cider, loaf, soak, spring, cheese, syrup, butt..."
4,4,Homemade Mac and Cheese Casserole,"['8 ounces whole wheat rotini pasta', '3 cups ...",Preheat oven to 350 degrees F. Line a 2-quart ...,YCnbhplMgiraW4rUXcybgSEZinSgljm,8 ounces whole wheat rotini pasta; 3 cups fres...,13,"olive, Bundt, ginger, inches, dash, Remove, ba..."
...,...,...,...,...,...,...,...,...
122906,122906,Summer Corn Salad,"['4 ears fresh corn', '2 heads Belgian endive'...",Watch how to make this recipe.\nPreheat a gril...,G9XPV1I0nULIbwS76f..0AIq2DsfI3u,4 ears fresh corn; 2 heads Belgian endive; 2 t...,10,"muffins, confectioners, overripe, spatula, spe..."
122907,122907,Zucchini Stuffed Tomatoes,"['4 large plum tomatoes', 'Salt and sugar', '1...",Preheat the broiler. Cut the tomatoes in 1/2 c...,nA7V5iQqaFcIEQQ0/MKqVdUioieXjPu,4 large plum tomatoes; Salt and sugar; 1 1/2 p...,9,NaN
122908,122908,Pepper Pasta Quick Cook,"['3 tablespoons olive oil', '2 tablespoons uns...",Heat the oil and butter in a large skillet ove...,NaN,3 tablespoons olive oil; 2 tablespoons unsalte...,17,"yogurt, pan, Drop, extract, rhubarb, crumbs, b..."
122909,122909,Chocolate Cake with Armagnac Ice Cream,"['8 ounces butter', '8 ounces bittersweet choc...",Preheat oven to 350 degrees. On the top half o...,NaN,8 ounces butter; 8 ounces bittersweet chocolat...,8,"boiling, raisins, walnuts, degrees, overripe, ..."


In [39]:
recipes.loc[:5,'tag_list']

0                                                  NaN
1    grease, paperline, hours, glaze, cranberries, ...
2                                                  NaN
3    cider, loaf, soak, spring, cheese, syrup, butt...
4    olive, Bundt, ginger, inches, dash, Remove, ba...
5    fluffy, wire, dough, topping, berries, bottom,...
Name: tag_list, dtype: object

In [44]:
recipes['tag_list'] = recipes['tag_list'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

# Concatenate lists of tags into a string for each document
recipes['tags'] = [' '.join(tags) for tags in recipes['tag_list']]

In [46]:
recipes.loc[:5,'tags']

0                                                     
1    grease  paperline  hours  glaze  cranberries  ...
2                                                     
3    cider  loaf  soak  spring  cheese  syrup  butt...
4    olive  Bundt  ginger  inches  dash  Remove  ba...
5    fluffy  wire  dough  topping  berries  bottom ...
Name: tags, dtype: object

In [50]:
# Creating TF-IDF Matrices and recalling text dependencies

# Assuming tokenized_text is your DataFrame with tokenized text
tokenized_text = pd.read_csv('tokenized_text.csv')

# Extracting the tokenized text from the DataFrame
tokenized_text = tokenized_text['0']

# Fit and transform your data
text_tfidf = vectorizer.fit_transform(tokenized_text)
title_tfidf = vectorizer.transform(recipes['title'])
# text_tfidf    <== Variable with recipe ingredients and instructions
tags_tfidf = vectorizer.transform(recipes['tags'])
# recipes   <== DataFrame; For indexing and printing recipes

# Query Similarity Weights
w_title = .2
w_text = .3
w_categories = .5

In [51]:
def qweight_array(query_length, qw_array = [1]):
    '''Returns descending weights for ranked query ingredients'''
    if query_length > 1:
        to_split = qw_array.pop()
        split = to_split/2
        qw_array.extend([split, split])
        return qweight_array(query_length - 1, qw_array)
    else:
        return np.array(qw_array)

def ranked_query(query):
    '''Called if query ingredients are ranked in order of importance.
    Weights and adds each ranked query ingredient vector.'''
    query = [[q] for q in query]      # place words in seperate documents
    q_vecs = [vectorizer.transform(q) for q in query] 
    qw_array = qweight_array(len(query),[1])
    q_weighted_vecs = q_vecs * qw_array
    q_final_vector = reduce(np.add,q_weighted_vecs)
    return q_final_vector

def overall_scores(query_vector):
    '''Calculates Query Similarity Scores against recipe title, instructions, and keywords.
    Then returns weighted averages of similarities for each recipe.'''
    final_scores = title_tfidf*query_vector.T*w_title
    final_scores += text_tfidf*query_vector.T*w_text
    final_scores += tags_tfidf*query_vector.T*w_categories
    return final_scores

def print_recipes(index, query, recipe_range):
    '''Prints recipes according to query similary ranks'''
    print('Search Query: {}\n'.format(query))
    for i, index in enumerate(index, recipe_range[0]):
        print('Recipe Rank: {}\t'.format(i+1),recipes.loc[index, 'title'],'\n')
        print('Ingredients:\n{}\n '.format(recipes.loc[index, 'ingredient_text']))
        print('Instructions:\n{}\n'.format(recipes.loc[index, 'instructions']))
        
def Search_Recipes(query, query_ranked=False, recipe_range=(0,3)):
    '''Master Recipe Search Function'''
    if query_ranked == True:
        q_vector = ranked_query(query)
    else:
        q_vector = vectorizer.transform([' '.join(query)])
    recipe_scores = overall_scores(q_vector)
    sorted_index = pd.Series(recipe_scores.toarray().T[0]).sort_values(ascending = False)[recipe_range[0]:recipe_range[1]].index
    return print_recipes(sorted_index, query, recipe_range)
    

In [52]:
query = ['cinnamon', 'cream', 'banana']
Search_Recipes(query, query_ranked=True, recipe_range=(0,3))

Search Query: ['cinnamon', 'cream', 'banana']

Recipe Rank: 1	 Cinnamon Ice Cream 

Ingredients:
2 cups half-and-half; 2 cups heavy cream; 1/2 vanilla bean, split lengthwise; 1 cinnamon stick; 3/4 teaspoon ground cinnamon; 9 egg yolks; 3/4 cup sugar
 
Instructions:
In a saucepan over medium heat, heat the half-and-half, cream, vanilla, cinnamon stick and ground cinnamon, whisking occasionally to make sure the mixture doesn?t burn or stick to the bottom of the pan. When the cream mixture reaches a fast simmer (do not let it boil), turn off the heat and let the flavors infuse for 10 minutes.
Whisk together the egg yolks and the sugar. In a thin stream, whisk half of the cream mixture into the egg yolk mixture. Then pour the egg-cream mixture back into the saucepan containing the rest of the cream mixture. Heat over medium heat, stirring constantly with a wooden spoon. At 160 degrees, the mixture will give off a puff of steam. When the mixture reaches 180 degrees it will be thickened and 

In [53]:
# Test Rank
query = ['wine', 'cilantro','butter']
Search_Recipes(query, query_ranked=False, recipe_range=(0,3))

Search Query: ['wine', 'cilantro', 'butter']

Recipe Rank: 1	 Cilantro Butter 

Ingredients:
1 stick unsalted butter, slightly softened; 6 cloves garlic, coarsely chopped; 1/4 cup fresh cilantro leaves; 1 to 2 teaspoons fresh lime juice; Kosher salt and freshly ground pepper
 
Instructions:
Combine the butter, garlic, cilantro and 1 teaspoon lime juice in a food processor or with a mixer until smooth. Season with salt and pepper, and add more lime juice if needed.
Photographs by Steve Giralt

Recipe Rank: 2	 Cilantro Oil 

Ingredients:
2 cups packed cilantro leaves; 1 cup lighttasting olive oil
 
Instructions:
Bring a large saucepan of water to a boil. Add the cilantro, making sure to push all the leaves under the boiling water. Blanch the cilantro for 5 seconds. Drain and immediately plunge into a bowl of ice water. Drain well and squeeze out all liquid.
In a blender, puree the cilantro with the olive oil. Strain the puree through 6 layers of cheesecloth or a paper coffee filter and p

In [54]:
# Test 
query = ['jelly','wine']
Search_Recipes(query, query_ranked=False, recipe_range=(0,3))

Search Query: ['jelly', 'wine']

Recipe Rank: 1	 Wine Jelly 

Ingredients:
3 1/2 cups wine; 1/2 cup fresh lemon juice; 1 (2 ounce) package dry pectin; 4 1/2 cups white sugar
 
Instructions:
Combine wine, lemon juice, and pectin in a large saucepot. Bring to a boil, stirring frequently. Add sugar, stirring until dissolved. Return to a rolling boil. Boil hard 1 minute, stirring constantly. Remove from heat. Skim foam off top, if necessary.
Ladle hot jelly into hot, sterilized jars, leaving 1/2 inch headspace. Tighten 2 piece lids. Process for 5 minutes in boiling water bath.


Recipe Rank: 2	 Pepper Jelly Appetizer  

Ingredients:
1 (8 ounce) package cream cheese, softened; 1/2 cup mild pepper jelly
 
Instructions:
Spread jelly over the block of cream cheese.


Recipe Rank: 3	 Cheesy Pepper Jelly 

Ingredients:
8 ounces cream cheese, at room temperature; 6 ounces pepper jelly; Olive oil and black pepper crackers, for serving
 
Instructions:
In a bowl mix the cream cheese and pepper jelly

In [55]:
query = ['pepper','apple','pork']
Search_Recipes(query, query_ranked=False, recipe_range=(0,3))

Search Query: ['pepper', 'apple', 'pork']

Recipe Rank: 1	 Apple Butter Pork Loin 

Ingredients:
2 (2 pound) boneless pork loin roast; seasoning salt to taste; 2 cups apple juice; 1/2 cup apple butter; 1/4 cup brown sugar; 2 tablespoons water; 1/4 teaspoon ground cinnamon; 1/4 teaspoon ground cloves
 
Instructions:
Preheat the oven to 350 degrees F (175 degrees C).
Season the pork loins with seasoning salt, and place them in a 9x13 inch baking dish or small roasting pan. Pour apple juice over the pork, and cover the dish with a lid or aluminum foil.
Bake for 1 hour in the preheated oven. While the pork is roasting, mix together the apple butter, brown sugar, water, cinnamon, and cloves. Remove pork roasts from the oven, and spread with apple butter mixture.
Cover, and return to the oven for 2 hours, or until fork-tender.


Recipe Rank: 2	 Shredded Apple Pork 

Ingredients:
1 cup MUSSELMAN'S® Apple Butter; 1/4 cup MUSSELMAN'S® Apple Cider Vinegar; 1/4 cup brown sugar; 2 tablespoons garl

In [56]:
recipes['tags'][122894]

'baking  muffin  dough  frothy  cutter  sifter  mixtures  squares  corners  line  cardamom  lumps  wire  base  Bundt  margarine  nutmeg  peaches  grapeseed  fold  sheet  seed  blueberry  half  minutesbaking  muffin  dough  frothy  cutter  sifter  mixtures  squares  corners  line  cardamom  lumps  wire  base  Bundt  margarine  nutmeg  peaches  grapeseed  fold  sheet  seed  blueberry  half  minutes'